In [14]:
import os
# os.chdir("..")

from torchvision.transforms import Lambda, Compose

from wf_vae.model import *
from wf_vae.dataset.transform import ToTensorVideo, CenterCropResizeVideo

from model.flameObj import *

os.getcwd()

'/nfs/horai.dgpsrv/ondemand28/harryscz/diffusion'

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
flamePath = "/scratch/ondemand28/harryscz/head_audio/head/code/flame/flame2023_no_jaw.npz"
sourcePath = "/scratch/ondemand28/harryscz/head_audio/head/data/vfhq-fit"
dataPath = [os.path.join(os.path.join(sourcePath, data), "fit.npz") for data in os.listdir(sourcePath)]
seqPath = "/scratch/ondemand28/harryscz/head/_-91nXXjrVo_00/fit.npz"

In [ ]:
head = Flame(flamePath, device="cuda")
head.loadSequence(dataPath[100])
seq = head.LSB(rotation=False)
head.renderVertsAnimation("test/orig3d.mp4", dist=1.2, resolution=512)
uvMesh = head.convertUV(rotation=False)
uv = head.get_uv_animation(uvMesh, savePath="test/origUV.mp4", sample_frames=800)
uv = uv[..., :3].permute(3,0,1,2).unsqueeze(0)
uv.shape

/nfs/horai.dgpsrv/ondemand28/harryscz/pytorch3d/pytorch3d/io/obj_io.py:551: UserWarning: Mtl file does not exist: /scratch/ondemand28/harryscz/head_audio/head/template.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


torch.Size([1, 3, 572, 256, 256])

In [60]:
model_cls = ModelRegistry.get_model("WFVAE")
vae = model_cls.from_pretrained("/scratch/ondemand28/harryscz/other/WF-VAE/weight")
vae = vae.to(device="cuda", dtype=torch.float32)

In [61]:
with torch.no_grad():
    z = vae.encode(uv.to("cuda")) # input shape : C, T, H, W
    latents = z.latent_dist.sample()
    recon = vae.decode(latents).sample
    
uv.shape, recon.shape

(torch.Size([1, 3, 572, 256, 256]), torch.Size([1, 3, 569, 256, 256]))

In [69]:
head.sampleFromUV(uv[0].permute(1,2,3,0), savePath="test/sampleOrig.mp4")
sampleRecon = head.sampleFromUV(recon[0].permute(1,2,3,0), savePath="test/sampleRecon.mp4")
head.sampleTo3D(sampleRecon, savePath="test/3dRecon.mp4", dist=1.2, resolution=512)

tensor([[[-0.0889, -0.2981, -3.0189],
         [-0.0889, -0.2981, -3.0189],
         [-0.0873, -0.2955, -3.0163],
         ...,
         [-0.3365, -0.2343, -2.8669],
         [-0.3365, -0.2343, -2.8669],
         [-0.3365, -0.2343, -2.8669]],

        [[-0.0882, -0.2984, -3.0192],
         [-0.0882, -0.2984, -3.0192],
         [-0.0871, -0.2950, -3.0177],
         ...,
         [-0.3346, -0.2355, -2.8653],
         [-0.3346, -0.2355, -2.8653],
         [-0.3346, -0.2355, -2.8653]],

        [[-0.0883, -0.2971, -3.0188],
         [-0.0883, -0.2971, -3.0188],
         [-0.0872, -0.2954, -3.0171],
         ...,
         [-0.3355, -0.2327, -2.8662],
         [-0.3355, -0.2327, -2.8662],
         [-0.3355, -0.2327, -2.8662]],

        ...,

        [[-0.0864, -0.2971, -3.0191],
         [-0.0864, -0.2971, -3.0191],
         [-0.0857, -0.2954, -3.0170],
         ...,
         [-0.3375, -0.2314, -2.8640],
         [-0.3375, -0.2314, -2.8640],
         [-0.3375, -0.2314, -2.8640]],

        [[

In [70]:
torch.abs( uv[:, :, :569, ...] - recon )

tensor([[[[[1.6865e-04, 9.7090e-04, 4.3264e-04,  ..., 1.3070e-03,
            7.4340e-04, 2.8167e-03],
           [2.4987e-03, 3.2180e-03, 7.6309e-04,  ..., 3.7611e-03,
            4.7899e-03, 4.5788e-03],
           [2.5387e-03, 1.3307e-04, 3.2893e-03,  ..., 1.4264e-03,
            2.1790e-03, 4.0027e-04],
           ...,
           [5.7054e-04, 7.8136e-04, 3.0449e-04,  ..., 3.9802e-03,
            4.4026e-03, 4.6333e-03],
           [1.8741e-03, 1.2329e-03, 2.4596e-04,  ..., 5.3759e-03,
            3.9098e-03, 3.1916e-03],
           [7.3138e-03, 3.2294e-04, 1.5045e-03,  ..., 4.8071e-03,
            8.3843e-03, 3.8617e-03]],

          [[3.3678e-03, 1.1796e-04, 2.2764e-03,  ..., 5.2939e-04,
            1.9143e-03, 4.1960e-04],
           [1.8397e-04, 1.9309e-04, 1.2939e-03,  ..., 5.4584e-04,
            2.4522e-03, 7.6990e-04],
           [2.5138e-03, 1.0153e-03, 1.8910e-03,  ..., 6.1144e-04,
            2.7952e-03, 2.2616e-03],
           ...,
           [1.7604e-03, 3.2732e-03, 2.7

In [72]:
latents.shape

torch.Size([1, 16, 143, 32, 32])